# 1. 한국도로공사 고속도로 교통사고 데이터 불러오기

## 1.1. 실습 개요

공공데이터포털에서 제공하는 **OpenAPI**를 이용하여 "한국도로공사_고속도로 교통사고 상세현황" 데이터를 수집하고, `pandas`를 사용해 **CSV 파일로 저장**하는 과정을 학습합니다.

## 1.2. 학습 목표

1. REST API의 기본 개념 이해하기
2. `requests` 라이브러리를 활용한 API 호출 방법 익히기
3. JSON 응답 데이터를 DataFrame으로 변환하기
4. CSV 파일로 저장하기

## 1.3. 사전 준비

- 공공데이터포털 회원가입: https://www.data.go.kr
- 데이터 활용신청 및 API Key 발급: https://www.data.go.kr/data/15145192/fileData.do

In [ ]:
# ============================================
# 2. 라이브러리 임포트 및 API 설정
# ============================================

import requests  # HTTP 요청을 보내기 위한 라이브러리
import pandas as pd  # 데이터 처리를 위한 라이브러리
import math  # 수학 함수 (올림 계산용)

# --------------------------------------------
# 2.1. API 서비스 키 설정
# --------------------------------------------
# - 공공데이터포털에서 발급받은 일반인증키를 입력합니다.
# - 각자 본인의 API Key로 변경해야 합니다.
SERVICE_KEY = "자신의 api key 입력하기"

# --------------------------------------------
# 2.2. API 엔드포인트 URL 설정
# --------------------------------------------
# - BASE_URL: 공공데이터포털 API의 기본 주소
# - ENDPOINT: 고속도로 교통사고 데이터의 고유 경로
BASE_URL = "https://api.odcloud.kr/api"
ENDPOINT = "/15145192/v1/uddi:d7c43b54-b689-4ef1-836b-9eba4eb98066"
URL = BASE_URL + ENDPOINT

# --------------------------------------------
# 2.3. API 호출 파라미터 설정
# --------------------------------------------
# - serviceKey: 인증을 위한 서비스 키
# - page: 요청할 페이지 번호 (1부터 시작)
# - perPage: 한 페이지당 가져올 데이터 건수 (최대 1000)
# - returnType: 응답 형식 (JSON)
PER_PAGE = 1000
params = {
    "serviceKey": SERVICE_KEY,
    "page": 1,
    "perPage": PER_PAGE,
    "returnType": "JSON",
}

# --------------------------------------------
# 2.4. API 테스트 호출
# --------------------------------------------
# - requests.get(): GET 방식으로 HTTP 요청을 보내는 함수
# - raise_for_status(): 응답 코드가 4xx/5xx이면 예외 발생
resp = requests.get(URL, params=params)
resp.raise_for_status()

# - resp.json(): JSON 응답을 파이썬 딕셔너리로 변환
data = resp.json()
data.keys()

In [ ]:
# ============================================
# 3. 전체 데이터 건수 및 페이지 수 확인
# ============================================

# - totalCount: API가 제공하는 전체 데이터 건수
# - math.ceil(): 올림 함수 (4.7 → 5)
# - 전체 데이터를 perPage 단위로 나누어 총 페이지 수 계산
total_count = data["totalCount"]
total_pages = math.ceil(total_count / PER_PAGE)

print("전체 데이터 수:", total_count)
print("총 페이지 수:", total_pages)

In [ ]:
# ============================================
# 4. 전체 페이지 순회하여 데이터 수집
# ============================================

# - range(1, total_pages + 1): 1부터 total_pages까지 순회
# - 각 페이지마다 API를 호출하여 데이터를 가져옴
# - extend(): 리스트에 여러 요소를 한 번에 추가

all_rows = []
for page in range(1, total_pages + 1):
    params["page"] = page
    resp = requests.get(URL, params=params)
    resp.raise_for_status()
    js = resp.json()
    all_rows.extend(js["data"])
    
print(f"수집 완료: 총 {len(all_rows)}건")

In [ ]:
# ============================================
# 5. DataFrame 변환 및 확인
# ============================================

# - pd.DataFrame(): 리스트/딕셔너리를 표 형태의 DataFrame으로 변환
# - shape: (행 개수, 열 개수) 튜플 반환
df = pd.DataFrame(all_rows)
print("최종 DataFrame 크기:", df.shape)

In [ ]:
# ============================================
# 6. 데이터 미리보기
# ============================================

# - head(n): 상위 n개 행 출력 (기본값: 5)
df.head(5)

In [ ]:
# ============================================
# 7. 컬럼 순서 정렬
# ============================================

# - 공공데이터포털의 파일 형식과 동일하게 컬럼 순서 변경
# - df[컬럼리스트]: 원하는 순서로 컬럼 재배치
print("순서 변경 전:", df.columns.tolist())

df = df[["사고년도", "사고일자", "사고시각", "노선명", "사고발생이정", "방향", "사망", "부상", "원인"]]

print("순서 변경 후:", df.columns.tolist())

In [ ]:
# ============================================
# 8. CSV 파일로 저장
# ============================================

# - to_csv(): DataFrame을 CSV 파일로 저장
# - index=False: 행 인덱스를 파일에 포함하지 않음
# - encoding="utf-8-sig": 한글이 깨지지 않도록 BOM이 포함된 UTF-8 인코딩
# - 주의: 코랩은 세션 기반이므로 세션 종료 시 파일이 사라집니다.

csv_path = "/content/highway_accident_full.csv"
df.to_csv(csv_path, index=False, encoding="utf-8-sig")

print(f"저장 완료: {csv_path}")

# 9. 다음 단계 안내

## 9.1. 이번 노트북에서 배운 내용

1. REST API를 통한 공공데이터 수집 방법
2. JSON 데이터를 DataFrame으로 변환하는 방법
3. CSV 파일로 저장하는 방법

## 9.2. 다음 노트북에서 학습할 내용

다음 노트북 `1_2_데이터_시각화_및_기본적_통계량.ipynb`에서는:

1. 데이터 기본 정보 확인 (shape, dtypes, info)
2. 결측치 분석
3. 기초 통계량 확인 (describe)
4. 데이터 시각화 (matplotlib, seaborn)

# 10. 참고: 주요 개념 정리

## 10.1. REST API

REST(Representational State Transfer)는 웹에서 자원을 URL로 표현하고, HTTP 메서드로 행동을 정의하는 아키텍처 스타일입니다.

| HTTP 메서드 | 설명 | 예시 |
|-------------|------|------|
| GET | 데이터 조회 | 사고 데이터 목록 조회 |
| POST | 데이터 생성 | 새로운 데이터 등록 |
| PUT/PATCH | 데이터 수정 | 기존 데이터 업데이트 |
| DELETE | 데이터 삭제 | 데이터 삭제 |

## 10.2. Query Parameter vs Path Parameter

| 구분 | 설명 | 예시 |
|------|------|------|
| Path Parameter | URL 경로에 포함된 고유 식별자 | `/users/123`, `/products/5567` |
| Query Parameter | URL 뒤에 `?`로 시작하는 필터/옵션 값 | `?page=1&perPage=1000` |

## 10.3. JSON (JavaScript Object Notation)

- 키-값 쌍으로 구성된 텍스트 기반 데이터 형식
- API 응답의 표준 포맷으로 널리 사용
- Python에서는 딕셔너리(dict)와 리스트(list)로 변환됨

```json
{
  "name": "홍길동",
  "age": 30,
  "skills": ["python", "pandas"]
}
```

## 10.4. CSV (Comma-Separated Values)

- 쉼표(,)로 구분된 텍스트 기반 표 데이터 형식
- Excel, Python, R 등 대부분의 도구에서 지원
- 단순하고 가벼워 데이터 교환에 널리 사용